In [ ]:
# display, transform, read, split ...
import numpy as np
import cv2 as cv
import os
import splitfolders
import matplotlib.pyplot as plt

# tensorflow
import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

### Step 2 - Data preprocessing

To use your data (images), you have to pre-process them. 

#### 1. Visualize dataset images

The first step is to display an image of each class to see what it looks like.

Here, there is **5 classes** (for 5 flower types).

In [ ]:
# daisy
img_daisy = image.load_img("/workspace/data/daisy/100080576_f52e8ee070_n.jpg")
img_daisy

In [ ]:
# dandelion
img_dandelion = image.load_img("/workspace/data/dandelion/10043234166_e6dd915111_n.jpg")
img_dandelion

In [ ]:
# roses
img_roses = image.load_img("/workspace/data/roses/10090824183_d02c613f10_m.jpg")
img_roses

In [ ]:
# sunflowers
img_sunflowers = image.load_img("/workspace/data/sunflowers/1008566138_6927679c8a.jpg")
img_sunflowers

In [ ]:
# tulips
img_tulips = image.load_img("/workspace/data/tulips/100930342_92e8746431_n.jpg")
img_tulips

#### 2. Split data to use a ResNet model

By using a **ResNet** model, your dataset has to be split as follow:

<img src="attachment:016f5918-d35a-4496-83a9-42843bbcd4fd.png"  width=400>

If this is not the case for your image classification dataset, follow the steps below.

1. Have a separate dataset as follows: **one folder per class**.

<img src="attachment:6cb999f7-3dfb-41df-a302-5115b27cb719.png" width=800>

2. Then play the following cell to split the dataset into training, validation and test sets.

In [ ]:
# split data in a new folder named data-split
splitfolders.ratio("/workspace/data", output="/workspace/data-split", seed=1337, ratio=(0.7, 0.2, 0.1), group_prefix=None, move=False)

Your data should, now, be split as follows:

<img src="attachment:713d2de3-0894-411b-8e55-0820b5768e93.png" width=900>

#### 3. Create Keras data generators

In [ ]:
datagen = ImageDataGenerator()

In [ ]:
# define classes name
class_names = ['daisy','dandelion','roses','sunflowers','tulips']

In [ ]:
# training data
train_generator = datagen.flow_from_directory( 
    directory="/workspace/data-split/train/", 
    classes = class_names,
    target_size=(224, 224),  
    batch_size=32, 
    class_mode="binary", 
)

In [ ]:
# validation data
valid_generator = datagen.flow_from_directory( 
    directory="/workspace/data-split/val/", 
    classes = class_names,
    target_size=(224, 224), 
    batch_size=32, 
    class_mode="binary", 
)

In [ ]:
# test data
test_generator = datagen.flow_from_directory( 
    directory="/workspace/data-split/test/", 
    classes = class_names,
    target_size=(224, 224), 
    batch_size=32, 
    class_mode="binary", 
)

### Step 3 - Build the model

The first step is to build the model, using **ResNet50**.

In [ ]:
# ResNet50 model
resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in resnet_50.layers:
    layer.trainable = False

In [ ]:
# build the entire model
x = resnet_50.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(5, activation='softmax')(x)
model = Model(inputs = resnet_50.input, outputs = predictions)

### Step 4 - Train the model

**Adam** optimizer is used to train the model over **10 epochs**. It is enough by using Transfer Learning.

The loss is calculated with the **sparse_categorical_crossentropy** function.

In [ ]:
# define training function
def trainModel(model, epochs, optimizer):
    batch_size = 32
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model.fit(train_generator, validation_data=valid_generator, epochs=epochs, batch_size=batch_size)

> Some warnings can appear, don't be afraid, you can execute the next steps of the notebook.

In [ ]:
# launch the training
model_history = trainModel(model = model, epochs = 10, optimizer = "Adam")

- Display **loss** curves:

In [ ]:
loss_train_curve = model_history.history["loss"]
loss_val_curve = model_history.history["val_loss"]
plt.plot(loss_train_curve, label = "Train")
plt.plot(loss_val_curve, label = "Validation")
plt.legend(loc = 'upper right')
plt.title("Loss")
plt.show()

- Display **accuracy** curves:

In [ ]:
acc_train_curve = model_history.history["accuracy"]
acc_val_curve = model_history.history["val_accuracy"]
plt.plot(acc_train_curve, label = "Train")
plt.plot(acc_val_curve, label = "Validation")
plt.legend(loc = 'lower right')
plt.title("Accuracy")
plt.show()

### Step 5 - Evaluate the model

The model is evaluated on test data.

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print("The test loss is: ", test_loss)
print("The best accuracy is: ", test_acc*100)

### Step 6 - Test the model on a new image

To test your model and predict which classes new images belong to, you can import sounds into a /workspace/data_test folder.

In [ ]:
img = tf.keras.preprocessing.image.load_img('/workspace/tulipe-test.jpeg', target_size=(224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.array([img_array]) 
img

In [ ]:
# generate predictions for samples
predictions = model.predict(img_array)
print(predictions)

In [ ]:
# generate argmax for predictions
class_id = np.argmax(predictions, axis = 1)
print(class_id)

In [ ]:
# transform classes number into classes name
class_names[class_id.item()]

### Step 7 - Save and export the model

>To save your model, you should create an other Object Storage container (with write rights) and mount it in your workspace (`saved_model` in this example).

You can now save your model in a dedicated folder.

In [ ]:
model.save('/workspace/saved_model/my_model')

In [ ]:
# my_model directory
%ls /workspace/saved_model/

In [ ]:
# contains an assets folder, saved_model.pb, and variables folder
%ls /workspace/saved_model/my_model

In [ ]:
model = tf.keras.models.load_model('/workspace/saved_model/my_model')
model.summary()

## Conclusion

**Transfer Learning** saves time by achieving better performance in fewer epochs. 

Train and test this model on your own dataset !